<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=58722b0b54bd59b6f374b879055c1950e77d637df0b0d0e3a307ba4b09e54a88
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-15 14:22:46
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 33.39 K (0.24%)
Current PnL: -26.14 L (-16.89%)
CY Booked + Current PnL: -11.85 L (-7.66%)
-------------------
Total profit:  1.44 L
Total loss:  -27.58 L
-------------------
Total Booked + Current PnL: 15.13 L (11.83%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.14%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.78 L (64.42%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.44%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.21,-16.56,19.96,0.09,16784.0,-16688.0,84089.0,95.09,44.0,M-SC,2.37,253.0,-0.99,0.60,10.47,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.14,-12.23,13.95,0.02,21373.0,-21345.0,153214.0,133.82,53.0,M-SC,15.34,234.0,-1.00,1.09,33.27,OX40N,NTT,PAINTS
82,VOLTAS,1530.00,0.57,8.35,10.47,19.69,21751.0,16008.0,207750.0,-2.18,56.0,X-MC,2.56,78.0,0.74,1.48,15.71,XY25,NTT,AC
50,MASFIN,398.61,0.62,-0.47,22.63,22.05,22068.0,-465.0,97515.0,-14.18,60.0,H-SC,7.18,164.0,-0.02,0.70,41.75,XR,ATH,FINANCE
43,ITC,452.00,0.51,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.72,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,12.00,-38.30,119.24,35.27,83541.0,-43488.0,70061.0,-51.71,53.0,L-SC,10.82,270.0,-0.52,0.50,71.48,XR,NTT,HOTELS
57,RAJOOENG,143.10,5.70,-34.55,113.30,39.61,76013.0,-35410.0,67090.0,-55.65,36.0,H-SC,28.07,135.0,-0.47,0.48,5.70,AR,ATH,MISC
44,JCHAC,2282.00,3.31,-35.39,54.75,-0.01,45211.0,-45227.0,82578.0,16791.30,33.0,M-SC,6.64,233.0,-1.00,0.59,5.69,OX40N,NTT,AC
79,VAIBHAVGBL,521.00,2.73,-25.12,114.71,60.77,156986.0,-45920.0,136855.0,57.58,51.0,H-SC,4.32,159.0,-0.29,0.98,23.25,XR,NTT,JEWELLERY
22,DIXON,18931.72,2.44,-8.74,38.14,26.07,67952.0,-17056.0,178165.0,-55.63,42.0,X-MC,8.32,56.0,-0.25,1.27,10.96,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.46,-17.81,121.04,81.67,93325.0,-16707.0,77103.0,6927.78,41.0,L-SC,12.96,271.0,-0.18,0.55,49.31,XR,NTT,CERAMICS
16,CERA,9475.0,-1.62,-32.35,83.14,23.89,98928.0,-56913.0,118990.0,-34.84,27.0,H-SC,10.30,157.0,-0.58,0.85,2.76,OX40N,NTT,CERAMICS
80,VALIANTORG,838.0,-1.29,-43.71,209.40,74.15,192834.0,-71516.0,92089.0,-265.94,47.0,H-SC,22.24,149.0,-0.37,0.66,18.04,XR,NTT,CHEMICALS
9,BAJAJHFL,181.5,-1.08,-18.97,91.86,55.47,181098.0,-46140.0,197146.0,-28.25,22.0,X-MC,12.50,64.0,-0.25,1.41,1.01,X40N,ATH,FINANCE
62,SATIN,274.0,-1.03,-21.31,92.94,51.83,202067.0,-58867.0,217417.0,-29.04,39.0,H-SC,2.83,146.0,-0.29,1.55,5.91,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,0.62,-0.47,22.63,22.05,22068.0,-465.0,97515.0,-14.18,60.0,H-SC,7.18,164.0,-0.02,0.7,41.75,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.05,-4.33,117.32,107.91,165714.0,-6394.0,141250.0,-22.17,50.0,M-SC,12.61,216.0,-0.04,1.01,5.32,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.14,-12.23,13.95,0.02,21373.0,-21345.0,153214.0,133.82,53.0,M-SC,15.34,234.0,-1.00,1.09,33.27,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.21,-16.56,19.96,0.09,16784.0,-16688.0,84089.0,95.09,44.0,M-SC,2.37,253.0,-0.99,0.60,10.47,OX40N,NTT,DURABLES
66,SIS,528.00,1.59,-24.21,60.00,21.26,50490.0,-26882.0,84150.0,1984.97,50.0,H-SC,4.86,166.0,-0.53,0.60,13.70,OX40N,NTT,MISC
47,KANSAINER,340.00,1.45,-24.33,49.95,13.47,101931.0,-65601.0,204066.0,-68.90,38.0,H-SC,4.74,158.0,-0.64,1.46,3.76,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.02,6.86,61.2,72.27,110818.0,11627.0,181075.0,-8.35,68.0,M-LC,2.21,99.0,0.1,1.29,13.46,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.02,6.86,61.20,72.27,110818.0,11627.0,181075.0,-8.35,68.0,M-LC,2.21,99.0,0.10,1.29,13.46,XR,NTT,IT
50,MASFIN,398.61,0.62,-0.47,22.63,22.05,22068.0,-465.0,97515.0,-14.18,60.0,H-SC,7.18,164.0,-0.02,0.70,41.75,XR,ATH,FINANCE
38,INDIAMART,4810.62,2.03,-2.13,111.22,106.72,134251.0,-2628.0,120708.0,-51.58,41.0,H-SC,2.65,139.0,-0.02,0.86,20.75,AR,ATH,MISC
25,FINCABLES,1641.55,2.05,-4.33,117.32,107.91,165714.0,-6394.0,141250.0,-22.17,50.0,M-SC,12.61,216.0,-0.04,1.01,5.32,OX40N,ATH,CABLES
37,IEX,219.00,-0.38,-3.18,53.51,48.63,103590.0,-6364.0,193590.0,-35.53,51.0,H-SC,14.04,136.0,-0.06,1.38,8.00,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.08,-18.97,91.86,55.47,181098.0,-46140.0,197146.0,-28.25,22.0,X-MC,12.50,64.0,-0.25,1.41,1.01,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.60,-8.87,40.57,28.11,74466.0,-17860.0,183550.0,-34.01,30.0,X-MC,12.66,55.0,-0.24,1.31,0.04,X40,ATH,APPARELS
75,TMPV,600.00,-0.30,-29.00,73.21,22.98,117416.0,-65515.0,160383.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.87,XY24,NTT,AUTO
23,DMART,5391.45,-0.64,-8.21,41.20,29.60,94391.0,-20496.0,229104.0,-21.75,32.0,X-LC,5.49,19.0,-0.22,1.63,12.18,X40N,ATH,FMCG
33,HINDUNILVR,2922.00,1.28,-10.94,27.63,13.67,63259.0,-28113.0,228950.0,-17.61,32.0,X-LC,2.18,14.0,-0.44,1.63,8.08,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.28,-0.77,25.45,24.48,63602.0,-1947.0,249909.0,-81.09,45.0,X-SC,0.70,86.0,-0.03,1.78,22.19,X40N,NTT,MISC
43,ITC,452.0,0.51,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.72,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.34,6.79,22.41,30.73,47147.0,13379.0,210384.0,-18.17,63.0,X-MC,2.12,75.0,0.28,1.50,20.43,X40,ATH,INSURANCE
33,HINDUNILVR,2922.0,1.28,-10.94,27.63,13.67,63259.0,-28113.0,228950.0,-17.61,32.0,X-LC,2.18,14.0,-0.44,1.63,8.08,XY25,NTT,FMCG
3,ACC,3906.0,0.27,-25.35,119.88,64.15,212955.0,-60311.0,177640.0,-56.31,35.0,X-SC,2.42,82.0,-0.28,1.27,0.27,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.60,-8.87,40.57,28.11,74466.0,-17860.0,183550.0,-34.01,30.0,X-MC,12.66,55.0,-0.24,1.31,0.04,X40,ATH,APPARELS
3,ACC,3906.00,0.27,-25.35,119.88,64.15,212955.0,-60311.0,177640.0,-56.31,35.0,X-SC,2.42,82.0,-0.28,1.27,0.27,XY24,BTT,CEMENT
75,TMPV,600.00,-0.30,-29.00,73.21,22.98,117416.0,-65515.0,160383.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.87,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-1.08,-18.97,91.86,55.47,181098.0,-46140.0,197146.0,-28.25,22.0,X-MC,12.50,64.0,-0.25,1.41,1.01,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.03,-38.08,118.34,35.20,94289.0,-48994.0,79676.0,4.40,34.0,X-SC,15.98,92.0,-0.52,0.57,1.21,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.39,-29.14,101.60,42.84,46797.0,-18946.0,46060.0,-53.72,44.0,X-SC,35.91,83.0,-0.40,0.33,4.21,XY24,NTT,MISC
59,RELAXO,1176.00,-0.12,-47.61,190.69,52.31,145029.0,-69105.0,76055.0,-44.25,42.0,X-SC,6.71,91.0,-0.48,0.54,1.80,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.03,-38.08,118.34,35.20,94289.0,-48994.0,79676.0,4.40,34.0,X-SC,15.98,92.0,-0.52,0.57,1.21,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.58,-4.34,29.70,24.08,36069.0,-5506.0,121444.0,-12.07,33.0,X-SC,5.23,89.0,-0.15,0.87,14.43,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.49,-16.35,71.34,43.32,97194.0,-26632.0,136240.0,-28.18,38.0,X-SC,4.62,90.0,-0.27,0.97,4.61,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.02,-11.44,36.29,20.71,121566.0,-43253.0,334984.0,-23.62,64.0,X-LC,4.66,1.0,-0.36,2.39,11.93,X40,ATH,IT
41,INFY,1972.00,0.59,9.44,22.66,34.24,77597.0,29552.0,342440.0,-14.93,64.0,X-LC,3.59,2.0,0.38,2.44,18.51,X40,NTT,IT
75,TMPV,600.00,-0.30,-29.00,73.21,22.98,117416.0,-65515.0,160383.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.87,XY24,NTT,AUTO
81,VBL,671.64,-0.74,-3.77,40.98,35.67,124556.0,-11899.0,303943.0,-15.14,55.0,X-LC,4.49,4.0,-0.10,2.17,9.57,X40N,ATH,FMCG
43,ITC,452.00,0.51,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.72,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.27,-37.53,112.73,32.89,54371.0,-28976.0,48231.0,-695.74,55.0,L-MC,5.87,259.0,-0.53,0.34,32.82,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.46,-17.81,121.04,81.67,93325.0,-16707.0,77103.0,6927.78,41.0,L-SC,12.96,271.0,-0.18,0.55,49.31,XR,NTT,CERAMICS
50,MASFIN,398.61,0.62,-0.47,22.63,22.05,22068.0,-465.0,97515.0,-14.18,60.0,H-SC,7.18,164.0,-0.02,0.70,41.75,XR,ATH,FINANCE
13,BSOFT,831.70,0.41,-22.10,93.55,50.79,102506.0,-31079.0,109574.0,0.12,67.0,H-SC,5.96,151.0,-0.30,0.78,27.58,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.14,-12.23,13.95,0.02,21373.0,-21345.0,153214.0,133.82,53.0,M-SC,15.34,234.0,-1.00,1.09,33.27,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.41,-22.10,93.55,50.79,102506.0,-31079.0,109574.0,0.12,67.0,H-SC,5.96,151.0,-0.30,0.78,27.58,XR,ATH,IT
84,WIPRO,420.00,-0.02,6.86,61.20,72.27,110818.0,11627.0,181075.0,-8.35,68.0,M-LC,2.21,99.0,0.10,1.29,13.46,XR,NTT,IT
41,INFY,1972.00,0.59,9.44,22.66,34.24,77597.0,29552.0,342440.0,-14.93,64.0,X-LC,3.59,2.0,0.38,2.44,18.51,X40,NTT,IT
32,HCLTECH,1922.01,0.60,9.23,14.19,24.73,37499.0,22338.0,264262.0,14.03,66.0,X-LC,6.72,7.0,0.60,1.88,25.88,X40,ATH,IT
1,ABB,7934.00,0.07,-3.10,50.22,45.57,127314.0,-8107.0,253512.0,-37.78,60.0,H-MC,4.58,120.0,-0.06,1.81,7.87,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.30
1,25,44.85
2,50,76.60


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.06
MC    30.90
LC    24.04
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.80
X40      22.94
X40N     13.01
XR        9.82
XY25      9.07
AR        9.05
OX40N     7.54
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.40
H-SC    23.90
X-LC    20.62
M-SC    11.68
X-SC     8.06
H-MC     4.82
L-SC     1.42
M-MC     1.34
M-LC     1.29
H-LC     1.12
L-LC     1.01
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.31,-7.43,41.98
IT,13.01,-15.12,73.01
FINANCE,9.55,-19.13,70.94
MISC,7.33,-26.50,78.25
ELECTRICAL,6.01,-11.23,51.97
PAINTS,5.85,-12.62,29.35
INSURANCE,4.78,-1.12,35.72
PHARMA,4.11,-1.86,33.92
AUTO,2.81,-32.70,78.71


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3163677.0,21
AR,1303122.0,10
XR,1285945.0,13
X40,1005273.0,14
X40N,860872.0,9
OX40N,734909.0,10
XY25,367402.0,6
SR,281707.0,2
MH,74925.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3629612.0,25
M-SC,1420291.0,15
X-MC,1404240.0,16
X-LC,833937.0,11
X-SC,789447.0,8
H-MC,402297.0,3
L-SC,262571.0,3
M-LC,110818.0,1
H-LC,75979.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1265347.0      6
           AR         893414.0      5
M-SC       XY24       815515.0      6
H-SC       XR         771103.0      7
X-MC       X40        490638.0      7
           XY24       399691.0      3
H-SC       OX40N      343116.0      4
X-MC       X40N       336100.0      4
X-LC       X40        326834.0      5
M-SC       OX40N      306088.0      5
X-SC       X40N       305825.0      3
           XY24       295821.0      3
H-SC       SR         281707.0      2
X-LC       X40N       218947.0      2
H-MC       AR         207828.0      2
           XY24       194469.0      1
X-LC       XY24       192834.0      2
X-SC       X40        187801.0      2
X-MC       XY25       177811.0      2
L-SC       XR         176866.0      2
M-SC       XR         172787.0      2
           AR         125901.0      2
M-LC       XR         110818.0      1
X-LC       XY25        95322.0      2
L-SC       OX40N       85705.0      1
H-LC       AR          75979.0      1
H-SC       MH          74925.0      1
M-MC       XY25        57015.0      1
L-MC       XR          54371.0      1
L-LC       XY25        37254.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
